In [ ]:
import os
from deepface import DeepFace
import cv2
from tqdm import tqdm
import numpy as np
import nest_asyncio
from qdrant_client import AsyncQdrantClient, models
from qdrant_client.models import Distance, VectorParams
import uuid

nest_asyncio.apply()

# Initialize Qdrant client
class QdrantConnection:
    def __init__(self) -> None:
        self.url = os.getenv('QDRANT_URL', 'http://localhost/')
        self.port = os.getenv('QDRANT_PORT', 6333)
        self.client = AsyncQdrantClient(url=self.url, port=self.port, timeout=60)

    async def create_collection(self, collection_name):
        try:
            result = await self.client.create_collection(
                collection_name=collection_name,
                vectors_config=VectorParams(
                    size=512, 
                    distance=Distance.COSINE  # Use COSINE for FaceNet512
                ),
                optimizers_config=models.OptimizersConfigDiff(
                    memmap_threshold=20000  # Use memory-mapped storage for large datasets
                ),
                hnsw_config=models.HnswConfigDiff(
                    m=32,  # Number of connections per node (balances speed & accuracy)
                    ef_construct=400,  # Improves recall during indexing
                    full_scan_threshold=10000,  # Avoid brute-force scans
                    on_disk=True  # Store index on disk for large-scale datasets
                ),
                quantization_config=models.ScalarQuantization(  # Optional: Enable quantization for memory efficiency
                    scalar=models.ScalarQuantizationConfig(
                        type="int8", quantile=0.99, always_ram=True
                    )
                )
            )
            return result
        except Exception as e:
            print(f"Error creating collection: {e}")

    async def batch_upload_points_to_collection(self, collection_name, payload):
        try:
            operation_info = await self.client.upsert(
                collection_name=collection_name,
                wait=True,
                points=models.Batch(
                    ids=payload.get('ids'),
                    vectors=payload.get('vectors'),
                    payloads=payload.get('payload')
                )
            )
            return operation_info
        except Exception as e:
            print(f"Error uploading points: {e}")
    
    async def query_collection(self, collection_name, query_vector: list):
        try:
            search_result = await self.client.query_points(
                collection_name=collection_name,
                query=query_vector,
                with_payload=True,
                limit=3
            )
            return search_result.points
        except Exception as e:
            print(f"Error querying collection: {e}")

qdrant_client = QdrantConnection()

# Create collection
await qdrant_client.create_collection("face-recognition")

# Extract embedding of the given image
given_image_path = "/path/to/your/given/image.jpg"
given_image_result = DeepFace.represent(img_path=given_image_path, model_name="Facenet512", detector_backend="mtcnn")
given_image_embedding = given_image_result[0].get('embedding')

# Upload given image embedding to Qdrant
payload = {
    'ids': [str(uuid.uuid4())],
    'vectors': [given_image_embedding],
    'payload': [{"file_path": given_image_path}]
}
await qdrant_client.batch_upload_points_to_collection('face-recognition', payload)

# Initialize video capture
video_path = "/path/to/your/video.mp4"
cap = cv2.VideoCapture(video_path)

# Check if video opened successfully
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

# Process video frames
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
pbar = tqdm(total=frame_count)
embeddings = []
file_path = []

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    try:
        # Save the frame as an image file
        img_path = "/tmp/frame.jpg"
        cv2.imwrite(img_path, frame)

        # Perform face recognition
        result = DeepFace.represent(img_path=img_path, model_name="Facenet512", detector_backend="mtcnn")
        for r in result:
            if r:
                embeddings.append(r.get("embedding"))
                file_path.append(img_path)

    except Exception as e:
        print(e)

    pbar.update(1)

pbar.close()
cap.release()

# Upload video frame embeddings to Qdrant
payload = {
    'ids': [str(uuid.uuid4()) for _ in range(len(embeddings))],
    'vectors': embeddings,
    'payload': [{"file_path": file} for file in file_path]
}
await qdrant_client.batch_upload_points_to_collection('face-recognition', payload)

# Query Qdrant to find frames matching the given image
res = await qdrant_client.query_collection('face-recognition', given_image_embedding)
print(res)

# Display recognized frames
for point in res:
    frame_path = point.payload['file_path']
    frame = cv2.imread(frame_path)
    cv2.imshow('Recognized Frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()